<a href="https://colab.research.google.com/github/MatthewAwesome/AIComposer/blob/master/AI_Composer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#An AI Composer via Machine Learning

The goal sought by this sequence of cell is straightforward: Use AI to compose a musical score that mashes up two songs of our choosing. 

Before beginning, you will need some MIDI files to play around with. You can download some [HERE](https://colinraffel.com/projects/lmd/#get)

The final output is a MIDI file that can be downloaded for subseqent mixing. Or, if you so prefer, the dowloaded MIDI file can be used to generated a sheet music score. A drag-and-drop web tool is available [HERE](https://solmire.com/miditosheetmusic/).

Snippets contained in the cells below expand upon [Magenta's MusicVAE tutorial Notebook](https://colab.research.google.com/github/magenta/magenta-demos/blob/master/colab-notebooks/MusicVAE.ipynb). 

The MusicVAE notebook provides additional examples and explanation.  

###Let's get after it. 

##Setting up the Environment

In [3]:
# Create a Magenta Environment: 

#@title Setup Environment
#@test {"output": "ignore"}

import glob

BASE_DIR = "gs://download.magenta.tensorflow.org/models/music_vae/colab2"

print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -q pyfluidsynth
!pip install -qU magenta

# Hack to allow python to pick up the newly-installed fluidsynth lib.
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library


print('Importing libraries and defining some helper functions...')
from google.colab import files
import magenta.music as mm
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel
import numpy as np
import os
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

# Necessary until pyfluidsynth is updated (>1.2.5).
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# A wrapper function to play a note sequence: 
def play(note_sequence):
  mm.play_sequence(note_sequence, synth=mm.fluidsynth)

# A function to interpolate two sequences: 
def interpolate(model, start_seq, end_seq, num_steps, max_length=32,
                assert_same_length=True, temperature=0.5,
                individual_duration=4.0):
  """Interpolates between a start and end sequence."""
  note_sequences = model.interpolate(
      start_seq, end_seq,num_steps=num_steps, length=max_length,
      temperature=temperature,
      assert_same_length=assert_same_length)

  print('Start Seq Reconstruction')
  play(note_sequences[0])
  print('End Seq Reconstruction')
  play(note_sequences[-1])
  print('Mean Sequence')
  play(note_sequences[num_steps // 2])
  print('Start -> End Interpolation')
  interp_seq = mm.sequences_lib.concatenate_sequences(
      note_sequences, [individual_duration] * len(note_sequences))
  play(interp_seq)
  mm.plot_sequence(interp_seq)
  return interp_seq if num_steps >= 3 else note_sequences[num_steps // 2]

def download(note_sequence, filename):
  mm.sequence_proto_to_midi_file(note_sequence, filename)
  files.download(filename)

print('Done')

Installing dependencies...
Importing libraries and defining some helper functions...
Done


##Loading the models.

In [2]:
# Grabbing the models: 
trio_models = {}
hierdec_trio_16bar_config = configs.CONFIG_MAP['hierdec-trio_16bar']
trio_models['hierdec_trio_16bar'] = TrainedModel(hierdec_trio_16bar_config, batch_size=1, checkpoint_dir_or_path=BASE_DIR + '/checkpoints/trio_16bar_hierdec.ckpt')
#
flat_trio_16bar_config = configs.CONFIG_MAP['flat-trio_16bar']
trio_models['baseline_flat_trio_16bar'] = TrainedModel(flat_trio_16bar_config, batch_size=4, checkpoint_dir_or_path=BASE_DIR + '/checkpoints/trio_16bar_flat.ckpt')

INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, HierarchicalLstmDecoder, and hparams:
{'max_seq_len': 256, 'z_size': 512, 'free_bits': 256, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 1, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [1024, 1024], 'enc_rnn_size': [2048, 2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'constant', 'sampling_rate': 0.0, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048, 2048]

Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
INFO:tensorflow:
Hierarchical Decoder:
  input length: 256
  level output lengths: [16, 16]

INFO:tensorflow:
Decoder Cells:
  units: [1024, 1024]



##Load up the MIDI files.

###Load the first track

In [4]:
# We use two cells here. One to load a first song. 
songs = []
songs.append(files.upload().values() or input_trio_midi_data)

Saving Band On The Run.mid to Band On The Run.mid


###Load the second track

In [5]:
# Another to load the second song.
songs.append(files.upload().values() or input_trio_midi_data)

Saving I Shot the Sheriff.2.mid to I Shot the Sheriff.2.mid


##Extract some trios: 

In the scheme of our experiment, this is a data cleaning step. We pare down our inputted songs to sequences of 16 bar 'grooves'.

The model(s) we loaded above will operate on these grooves. Interpolating them to create a 'mashup' 

Note: Trios may be un-extractable from some songs. These songs break the pipeline so to speak. If encountered, run the track loading cells again, loading different songs. 

In [6]:
all_trios = []
for i,s in enumerate(songs): 
  trio_input_seqs = [mm.midi_to_sequence_proto(m) for m in s]
  extracted_trios = []
  for ns in trio_input_seqs:
    extracted_trios.extend(
        hierdec_trio_16bar_config.data_converter.from_tensors(
            hierdec_trio_16bar_config.data_converter.to_tensors(ns)[1]))
  all_trios.append(extracted_trios)

## Trio Mashup Time! 

Here we interpolate, or 'mashup' the trios. This is performed via interpolation. 

You can also play with the Temperature slider. Any guesses as to what it does?

In [7]:
#@title Compute the reconstructions and mean of the two trios, selected from the previous cell.

# hierdec contains an LSTM hierarchy,
# flat does not. 

trio_interp_model = "hierdec_trio_16bar" #@param ["hierdec_trio_16bar", "baseline_flat_trio_16bar"]

song_one_trio =  1#@param {type:"integer"}
song_two_trio =  0#@param {type:"integer"}
start_trio = all_trios[0][song_one_trio]
end_trio = all_trios[1][song_two_trio]

temperature = 1.2 #@param {type:"slider", min:0.1, max:1.5, step:0.1}
trio_16bar_mean = interpolate(trio_models[trio_interp_model], start_trio, end_trio, num_steps=3, max_length=256, individual_duration=32, temperature=temperature)

Start Seq Reconstruction


End Seq Reconstruction


Mean Sequence


Start -> End Interpolation


##Download the MIDI file for some mixing fun!

In [8]:
download(trio_16bar_mean, 'your_awesome_mashup.mid')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##That's it! 

Again, this is a trimmed down version of the [MusicVAE Notebook ](https://colab.research.google.com/github/magenta/magenta-demos/blob/master/colab-notebooks/MusicVAE.ipynb) provided by the Magenta team.

Play around with theirs, and take a peek if you want more detail. 

#Stay Creative